In [1]:
mainland_bank = ['939']
mainland_insur = ['2318','2628']
mainland_realestate = ['688']
gambling = ['27','1928']
energy = ['883','386','857']
telecommunications = ['728','941']
Network = ['700','992']
local_insur = ['1299']
local_bank = ['5']
local_realestate = ['16']
Finance = ['388']
public_service = ['902']
car = ['2333']
congol = ['1']

strategy_list = ['0939.HK', '2318.HK', '2628.HK', '0688.HK', '0027.HK', '1928.HK', '0883.HK', '0386.HK', 
                 '0857.HK', '0728.HK', '0941.HK', '0700.HK', '0992.HK', '1299.HK', '0005.HK', '0016.HK', 
                 '0388.HK', '0902.HK', '2333.HK', '0001.HK']

namelist = []

for i in strategy_list:
    namelist.append(i.split('.')[0])

In [4]:
def signal_update(df):
    
    openp = df.loc[:,'Open'].tolist()[-3:]
    closep = df.loc[:,'Close'].tolist()[-3:]
    lowp = df.loc[:,'Low'].tolist()[-3:]
    highp = df.loc[:,'High'].tolist()[-3:]


    # BigYang and BigYin
    change = percentChange(openp[-1],closep[-1])
    if change >= 4.00 and change <= 5.00:
        df.at[(len(df)-1), 'BigYang']= 1
    else: df.at[(len(df)-1), 'BigYang']= 0
    if change <= -4.00 and change >= -5.00:
        df.at[(len(df)-1), 'BigYin']= 1
    else: df.at[(len(df)-1), 'BigYin']= 0
        

    # Goodeat and Badeat
    if closep[-2] < openp[-2] and closep[-1] > openp[-1] and closep[-1] > openp[-2] and closep[-2] > openp[-1]:
        df.at[(len(df)-1), 'Goodeat']= 1
    else: df.at[(len(df)-1), 'Goodeat']= 0
    if closep[-2] > openp[-2] and closep[-1] < openp[-1] and closep[-2] < openp[-1] and closep[-1] < openp[-2]:
        df.at[(len(df)-1), 'Badeat']= 1
    else: df.at[(len(df)-1), 'Badeat']= 0
        

    # Bull_harami and Bear_harami
    if closep[-2] < openp[-2] and closep[-1] > openp[-1] and closep[-2] < openp[-1] and closep[-1] < openp[-2]:
        df.at[(len(df)-1), 'Bull_harami']= 1
    else:  df.at[(len(df)-1), 'Bull_harami']= 0
    if closep[-2] > openp[-2] and closep[-1] < openp[-1] and closep[-2] > openp[-1] and closep[-1] > openp[-2]:
        df.at[(len(df)-1), 'Bear_harami']= 1
    else:  df.at[(len(df)-1), 'Bear_harami']= 0


    # Bull_reverse and Bear_reverse
    if ((closep[-1]-closep[-2])/closep[-2]) > 0.03 and ((closep[-2]-closep[-3])/closep[-3]) < -0.03:
        df.at[(len(df)-1), 'Bull_reverse']= 1
    else: df.at[(len(df)-1), 'Bull_reverse']= 0
    if ((closep[-1]-closep[-2])/closep[-2]) < -0.03 and ((closep[-2]-closep[-3])/closep[-3]) > 0.03:
        df.at[(len(df)-1), 'Bear_reverse']= 1
    else: df.at[(len(df)-1), 'Bear_reverse']= 0


    # sunrise and darkcloud
    if openp[-2] > closep[-2] and openp[-1] < closep[-1] and openp[-1] < closep[-2] and openp[-2] > closep[-1] and ((closep[-2]+openp[-2])/2) < closep[-1]:
        df.at[(len(df)-1), 'sunrise']= 1
    else: df.at[(len(df)-1), 'sunrise']= 0
    if openp[-2] < closep[-2] and openp[-1] > closep[-1] and openp[-2] < closep[-1] and openp[-1] > closep[-2] and ((closep[-2]+openp[-2])/2) > closep[-1]:
        df.at[(len(df)-1), 'darkcloud']= 1
    else: df.at[(len(df)-1), 'darkcloud']= 0


    # hammer and shootingstar
    if highp[-1] - closep[-1] <= closep[-1] - openp[-1] and closep[-1] > openp[-1] and (openp[-1] - lowp[-1]) >= 2*(closep[-1]-openp[-1]) and (closep[-1] - openp[-1])/openp[-1] > 0.005:
        df.at[(len(df)-1), 'hammer']= 1
    else: df.at[(len(df)-1), 'hammer']= 0
    if openp[-1] > closep[-1] and (highp[-1] - openp[-1]) >= (2*(openp[-1]-closep[-1])) and (closep[-1]-lowp[-1]) <= (openp[-1]-closep[-1]) and ((openp[-1]-closep[-1])/openp[-1]) > 0.005:
        df.at[(len(df)-1), 'shootingstar']= 1
    else: df.at[(len(df)-1), 'shootingstar']= 0


In [16]:
import urllib3
from bs4 import BeautifulSoup as bs
import pandas as pd
import datetime
import time

Be_rich = []
Be_rich_str = []

df_Long = pd.read_csv('/Users/ktchan33/Desktop/Python/Option_trade/strategy_trade/Backtest_21082017/Backtest_Long_21082017.csv')
df_Long = df_Long.set_index('Num')
df_Strangle = pd.read_csv('/Users/ktchan33/Desktop/Python/Option_trade/strategy_trade/Backtest_21082017/Backtest_Strangle_21082017.csv')
df_Strangle = df_Strangle.set_index('Num')

url = 'http://www.futunn.com/quote/stock?m=hk&code=0{}'

localpath = '/Users/ktchan33/Desktop/Python/Option_trade/strategy_trade/{}.HK.csv'

ii = 0

signal = ['BigYang','BigYin','Goodeat','Badeat','Bull_harami',
          'Bear_harami','Bull_reverse','Bear_reverse','sunrise',
          'darkcloud','shootingstar','hammer']

def percentChange(startPoint,currentPoint):
    return ((float(currentPoint-startPoint))/abs(startPoint))*100

def current_price(rows):
    data = []
    for each_row in rows:
        check_price01 = str(each_row).find('price01')
        if check_price01 != -1:
            dex = rows.index(each_row)
            data.append(['close',float(str(rows[dex]).split('>')[1].split('<')[0])])
            data.append(['high',float(str(rows[dex+3]).split('>')[1].split('<')[0])])
            data.append(['low',float(str(rows[dex+4]).split('>')[1].split('<')[0])])
            data.append(['open',float(str(rows[dex+5]).split('>')[1].split('<')[0])])
            break

    return data

for main in namelist:
    while ii < 5:
        http=urllib3.PoolManager()
        response =http.request('GET',url.format(main)) 
        rows = bs(response.data.decode('utf-8')).findAll('span')
        price_current = current_price(rows)

        if price_current is not None: 
            response.close()
            break
        else: 
            response.close()
            ii += 1

    df = pd.read_csv(localpath.format(main))


    today = datetime.datetime.now().strftime('%-d/%-m/%Y')
    df.at[len(df), 'Date'] =today
    
    if price_current[0][1] == 0.0 or price_current[1][1] == 0.0 or price_current[2][1] == 0.0 or price_current[3][1] == 0.0:
        continue
        
    for i in price_current:
        if i[0] == 'close': df.at[len(df)-1, 'Close']= i[1]
        elif i[0] == 'low': df.at[len(df)-1, 'Low']= i[1]   
        elif i[0] == 'high': df.at[len(df)-1, 'High']= i[1]  
        elif i[0] == 'open': df.at[len(df)-1, 'Open']= i[1]  

    
    signal_update(df)

    for i in signal:
        if df.loc[len(df)-1,i] == 1:
            callputdex = float(df_Long.loc[int(main),i].split(' ')[0])
            strdex = float(df_Strangle.loc[int(main),i].split(' ')[0])
            
            df_signal = pd.read_csv('/Users/ktchan33/Desktop/Python/Option_trade/strategy_trade/Backtest_21082017/{}_{}.csv'.format(main,i))
            df_signal = df_signal.set_index(['Descript'])
            upm = df_signal.loc['up3_median',:].tolist()
            downm = df_signal.loc['down3_median',:].tolist()
            
            if callputdex > 100 or callputdex < -100:
                if strdex > 450:
                    Be_rich.append([main,i,callputdex,strdex,upm,downm])
                elif strdex < 450:
                    Be_rich.append([main,i,callputdex,upm,downm])
                    
            if strdex > 500:
                Be_rich.append([main,i,callputdex,strdex,upm,downm])    

for trade in Be_rich:
    print (trade)
    print ("==================================================================================")
    
for trade in Be_rich_str:
    print (trade)
    print ("==================================================================================")
    
print ('Thats all')

IndexError: list index out of range

In [ ]:
print price_current

In [ ]:
print main

In [10]:
!echo $PATH

/Applications/JupyterLab.app/Contents/Resources/jlab_server/bin:/Applications/JupyterLab.app/Contents/Resources/jlab_server/condabin:/Library/Frameworks/Python.framework/Versions/3.8/bin:/Library/Frameworks/Python.framework/Versions/3.7/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:/node_modules/.bin:/Applications/JupyterLab.app/Contents/MacOS


In [11]:
!pip3 install bs4

Defaulting to user installation because normal site-packages is not writeable
  Using cached bs4-0.0.1.tar.gz (1.1 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.4/97.4 KB 6.0 MB/s eta 0:00:00
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1272 sha256=77d07318e9278cba52f98b38458506d4ae4febff2fdc0c9de5163af61fb8edb6
  Stored in directory: /Users/ktchan33/Library/Caches/pip/wheels/75/78/21/68b124549c9bdc94f822c02fb9aa3578a669843f9767776bca
Successfully built bs4
